In [4]:
import time as t
from IPython.display import clear_output
import numpy    as np
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt
import sklearn  as skl

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config

set_config(display='diagram') # Useful for display the pipeline

print("Pandas  ", pd.__version__)
print("Sklearn ", skl.__version__) # Try to use 0.24

Pandas   1.2.4
Sklearn  0.24.1


In [74]:
df_train = pd.read_csv('Datasets/train.csv')
df_test = pd.read_csv('Datasets/test.csv')

In [130]:
test_id = df_test.Id

In [76]:
#missing data
total = df_train.isnull().sum().sort_values(ascending=False)
percent = (df_train.isnull().sum()/df_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
PoolQC,1453,0.995205
MiscFeature,1406,0.963014
Alley,1369,0.937671
Fence,1179,0.807534
FireplaceQu,690,0.472603
LotFrontage,259,0.177397
GarageYrBlt,81,0.055479
GarageCond,81,0.055479
GarageType,81,0.055479
GarageFinish,81,0.055479


In [77]:
df_train =df_train.drop((missing_data[missing_data['Total'] > 100]).index,1)
df_train =df_train.drop(df_train.loc[df_train['Electrical'].isnull()].index)
df_train.isnull().sum().max() #just checking that there's no missing data missing...

81

In [114]:
 X = df_train.drop(['SalePrice','Id'], axis=1)
 y= df_train['SalePrice']
 X_test = df_test.drop(['Id'], axis=1)
 X_test_id = df_test['Id']

In [115]:
X['MSSubClass'] = X['MSSubClass'].astype(str)
X_test['MSSubClass'] = X_test['MSSubClass'].astype(str)

In [117]:
#Split Numerical and Categorical columns

num_cols, cat_cols = [], []

for col in X.columns:
    if X[col].dtype == object:
        cat_cols.append(col)
    else:
        num_cols.append(col)

print('There are', len(cat_cols), 'categorical columns.')
print('There are', len(num_cols), 'numerical columns.')

There are 39 categorical columns.
There are 34 numerical columns.


In [106]:
#cat_vars  = ['MSZoning','Neighborhood']       # x.select_dtypes(include=[object]).columns.values.tolist()
#num_vars  = ['OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']

cat_vars = cat_cols
num_vars = num_cols

print("\nNumerical features:\n", num_vars)
print("\nCategorical features:\n", cat_vars)


Numerical features:
 ['LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']

Categorical features:
 ['MSSubClass', 'MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'GarageType', 'GarageFinish', 'GarageQual', 'Ga

In [107]:
# YOUR CODE HERE
num_4_treeModels = pipeline.Pipeline(steps=[
  ('imputer', impute.SimpleImputer(strategy='mean', add_indicator=False))
])

cat_4_treeModels = pipeline.Pipeline(steps=[
  ('imputer', impute.SimpleImputer(strategy='constant', fill_value='missing')),
  ('ordinal', preprocessing.OrdinalEncoder(categories='auto', handle_unknown='use_encoded_value', unknown_value=99999999)) 
  # ('ordinal', preprocessing.OrdinalEncoder(categories='auto', handle_unknown='ignore'))
])

tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_4_treeModels, num_vars),
    ('cat', cat_4_treeModels, cat_vars),
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

tree_prepro

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 ['LotArea', 'OverallQual', 'OverallCond',
                                  'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
                                  'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
                                  'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
                                  'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
                                  'BsmtHalfBath', 'FullBath', 'HalfBath',
                                  'BedroomAbvGr', 'KitchenAbvGr',
                                  'To...
                                 ['MSSubClass', 'MSZoning', 'Street',
                                  'LotShape', 'LandContour', 'Utilities',
                                  'LotConfig', 'LandSlope', 'Neighborhood',
                                  'Condition1', 'Condition2', 'BldgType',
                                  'HouseStyle', 'RoofStyle', 'RoofMatl',
                                  'Exterior1st', 'Exterior2nd', 'MasVnrType',
                                  'ExterQual', 'ExterCond', 'Foundation',
                                  'BsmtQual', 'BsmtCond', 'BsmtExposure',
                                  'BsmtFinType1', 'BsmtFinType2', 'Heating',
                                  'HeatingQC', 'CentralAir', 'Electrical', ...])])

In [108]:
num_4_multi_models = pipeline.Pipeline(steps=[
  ('imputer', impute.SimpleImputer(strategy='mean', add_indicator=False)),
  ('scaler',  preprocessing.QuantileTransformer())
])

cat_4_treeModels = pipeline.Pipeline(steps=[
  ('imputer', impute.SimpleImputer(strategy='constant', fill_value='missing')),
  ('onehot', preprocessing.OneHotEncoder(categories='auto', handle_unknown='ignore')) 
]) #handle_unknown='use_encoded_value', unknown_value=99999999 --> what is this ?

multi_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_4_multi_models, num_vars),
    ('cat', cat_4_treeModels, cat_vars),
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

multi_prepro

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler',
                                                  QuantileTransformer())]),
                                 ['LotArea', 'OverallQual', 'OverallCond',
                                  'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
                                  'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
                                  'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
                                  'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
                                  'BsmtHalfBath', 'FullBath', 'HalfBath',...
                                 ['MSSubClass', 'MSZoning', 'Street',
                                  'LotShape', 'LandContour', 'Utilities',
                                  'LotConfig', 'LandSlope', 'Neighborhood',
                                  'Condition1', 'Condition2', 'BldgType',
                                  'HouseStyle', 'RoofStyle', 'RoofMatl',
                                  'Exterior1st', 'Exterior2nd', 'MasVnrType',
                                  'ExterQual', 'ExterCond', 'Foundation',
                                  'BsmtQual', 'BsmtCond', 'BsmtExposure',
                                  'BsmtFinType1', 'BsmtFinType2', 'Heating',
                                  'HeatingQC', 'CentralAir', 'Electrical', ...])])

In [92]:
from sklearn.tree          import DecisionTreeRegressor
from sklearn.ensemble      import RandomForestRegressor
from sklearn.ensemble      import ExtraTreesRegressor
from sklearn.ensemble      import AdaBoostRegressor
from sklearn.ensemble      import GradientBoostingRegressor
from catboost import CatBoostRegressor

In [93]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import BayesianRidge

In [109]:

# YOUR CODE HERE
#we dont need scaling/norm for these models
#ordinal encoding (sometimes binary) for categorical 
tree_regressor = {
  "Decision Tree": DecisionTreeRegressor(),
  "Extra Trees":ExtraTreesRegressor(),
  "Random Forest":RandomForestRegressor(),
  "AdaBoost":AdaBoostRegressor(),
  "GBM_regressor":GradientBoostingRegressor(),
  #"HGB_regressor":HistGradientBoostingRegressor(),
  "CatBoost":CatBoostRegressor()
}

#we need to do scaling/norm for these models
#one hot encoding for categorical
mult_regressor= {
    "Linear_regression": LinearRegression(),
    "Ridge_regessor": Ridge(),
    "SVM_regressor":SVR(),
    "MLP_regressor":MLPRegressor(),
    "SDG_regressor": SGDRegressor(),
    "KNN_regressor": KNeighborsRegressor(),
    #"Br_regressor": BayesianRidge(),
}


tree_models_regressor = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_regressor.items()}
multi_models_regressor = {name: pipeline.make_pipeline(multi_prepro, model) for name, model in mult_regressor.items()}


tree_models_regressor["Decision Tree"]

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['LotArea', 'OverallQual',
                                                   'OverallCond', 'YearBuilt',
                                                   'YearRemodAdd', 'MasVnrArea',
                                                   'BsmtFinSF1', 'BsmtFinSF2',
                                                   'BsmtUnfSF', 'TotalBsmtSF',
                                                   '1stFlrSF', '2ndFlrSF',
                                                   'LowQualFinSF', 'GrLivArea',
                                                   'BsmtFullBath',
                                                   'BsmtHalfBath', 'FullBath',
                                                   'HalfBa...
                                                   'LotConfig', 'LandSlope',
                                                   'Neighborhood', 'Condition1',
                                                   'Condition2', 'BldgType',
                                                   'HouseStyle', 'RoofStyle',
                                                   'RoofMatl', 'Exterior1st',
                                                   'Exterior2nd', 'MasVnrType',
                                                   'ExterQual', 'ExterCond',
                                                   'Foundation', 'BsmtQual',
                                                   'BsmtCond', 'BsmtExposure',
                                                   'BsmtFinType1',
                                                   'BsmtFinType2', 'Heating',
                                                   'HeatingQC', 'CentralAir',
                                                   'Electrical', ...])])),
                ('decisiontreeregressor', DecisionTreeRegressor())])

In [118]:
x_train, x_valid, y_train, y_valid = model_selection.train_test_split(X, y, test_size=0.2, random_state=0)

results = pd.DataFrame({'Model': [], 'MSE': [], 'RMSE': [],'RMSLE': [], 'Time': [],})

for name, pipe in tree_models_regressor.items():

    start_time = t.time()

    preds = pipe.fit(x_train,y_train).predict(x_valid)

    total_time = t.time() - start_time

    results = results.append({"Model": name,
                              "MSE":   metrics.mean_squared_error(y_valid, preds),
                              "RMSE":  metrics.mean_squared_error(y_valid, preds,squared=False),
                              'RMSLE': metrics.mean_squared_log_error(y_valid, abs(preds)),
                              "Time":  total_time},
                              ignore_index=True)


    results_ord = results.sort_values(by=['RMSLE'], ascending=True, ignore_index=True)
    results_ord.index += 1 
    clear_output()
    display(results_ord.style.bar(subset=['MSE', 'RMSE','RMSLE'], vmin=0, color='#5fba7d'))

,Model,MSE,RMSE,RMSLE,Time
1,CatBoost,572606540.232754,23929.198487,0.016208,9.553206
2,GBM_regressor,665023853.985669,25788.056421,0.019239,1.135984
3,Extra Trees,741721182.109568,27234.558599,0.020816,1.936171
4,Random Forest,702606002.315616,26506.716174,0.020935,2.178595
5,Decision Tree,1659270157.476027,40734.139950,0.044254,0.163617
6,AdaBoost,1354146198.774779,36798.725505,0.046199,0.674225


In [119]:
# Apply linear regression
x_train, x_valid, y_train, y_valid = model_selection.train_test_split(X, y, test_size=0.2, random_state=0)

results = pd.DataFrame({'Model': [], 'MSE': [], 'RMSE': [],'RMSLE': [], 'Time': [],})

for name, pipe in multi_models_regressor.items():

    start_time = t.time()

    preds = pipe.fit(x_train,y_train).predict(x_valid)

    total_time = t.time() - start_time

    results = results.append({"Model": name,
                              "MSE":   metrics.mean_squared_error(y_valid, preds),
                              "RMSE":  metrics.mean_squared_error(y_valid, preds,squared=False),
                              'RMSLE': metrics.mean_squared_log_error(y_valid, abs(preds)),
                              "Time":  total_time},
                              ignore_index=True)


    results_ord = results.sort_values(by=['RMSLE'], ascending=True, ignore_index=True)
    results_ord.index += 1 
    clear_output()
    display(results_ord.style.bar(subset=['MSE', 'RMSE','RMSLE'], vmin=0, color='#5fba7d'))

,Model,MSE,RMSE,RMSLE,Time
1,SDG_regressor,803217482.300166,28341.091763,0.025052,0.348099
2,Ridge_regessor,858714040.838553,29303.822973,0.028363,0.315928
3,KNN_regressor,1243899826.672055,35268.964071,0.031331,0.511733
4,Linear_regression,1022400145.669938,31974.992505,0.039768,0.783492
5,SVM_regressor,6269066796.842179,79177.438686,0.164569,1.300504
6,MLP_regressor,34561116382.751884,185906.203185,8.593819,7.609372


# Best Model

In [121]:
X_train_prepro = tree_prepro.fit_transform(x_train)
X_valid_prepro = tree_prepro.fit_transform(x_valid)

In [122]:
model = CatBoostRegressor()
model.fit(X_train_prepro,y_train)

Learning rate set to 0.040118
0:	learn: 77887.0410921	total: 17.3ms	remaining: 17.3s
1:	learn: 75942.0393252	total: 32.5ms	remaining: 16.2s
2:	learn: 73984.3148780	total: 43.7ms	remaining: 14.5s
3:	learn: 72279.1662496	total: 55.8ms	remaining: 13.9s
4:	learn: 70496.9066213	total: 72.9ms	remaining: 14.5s
5:	learn: 68883.2037820	total: 89.2ms	remaining: 14.8s
6:	learn: 67188.5176791	total: 105ms	remaining: 14.9s
7:	learn: 65637.6655261	total: 123ms	remaining: 15.3s
8:	learn: 64151.0985026	total: 137ms	remaining: 15.1s
9:	learn: 62766.8255793	total: 153ms	remaining: 15.1s
10:	learn: 61471.4770225	total: 168ms	remaining: 15.1s
11:	learn: 60102.7795682	total: 202ms	remaining: 16.6s
12:	learn: 58752.0341982	total: 234ms	remaining: 17.8s
13:	learn: 57433.9294691	total: 250ms	remaining: 17.6s
14:	learn: 56255.2924017	total: 264ms	remaining: 17.3s
15:	learn: 55114.3018156	total: 276ms	remaining: 17s
16:	learn: 54103.3970199	total: 296ms	remaining: 17.1s
17:	learn: 53002.5760329	total: 331ms	rem

In [123]:
y_pred = model.predict(X_valid_prepro)

In [125]:
print("Accuracy:\t", model.score(X_valid_prepro, y_valid)*100)
print("MSE : ",   metrics.mean_squared_error(y_valid, y_pred))

Accuracy:	 89.70107962034432
MSE :  625971514.4117012


# Submission

In [128]:
x_test_prepro = tree_prepro.fit_transform(X_test)
predictions = model.predict(x_test_prepro)

In [132]:
submission = pd.DataFrame({'Id': test_id, 'SalePrice':predictions})
filename = 'house_pricing_submission_2.csv'

submission.to_csv(filename,index=False)

print('Saved file: ' + filename)   

Saved file: house_pricing_submission_2.csv
